In [16]:
import pandas as pd
from datetime import datetime, timedelta

# Define input and output filenames
input_file = '/content/drive/MyDrive/Colab Notebooks/Historical Data Transformation/input.csv'
output_file = '/content/drive/MyDrive/Colab Notebooks/Historical Data Transformation/output.csv'
data = pd.read_csv(input_file)

# Convert date columns to datetime format
date_columns = ['Date of Joining', 'Date of Exit', 'Compensation 1 date', 'Compensation 2 date', 'Review 1 date', 'Review 2 date', 'Engagement 1 date', 'Engagement 2 date']
for column in date_columns:
    data[column] = pd.to_datetime(data[column])

# Initialize list to store transformed data
transformed_data = []

# Iterate through each row to transform the data
for _, row in data.iterrows():
    # Derive end date for each historical record
    if pd.isnull(row['Date of Exit']):
        end_date = datetime(2100, 1, 1)
    else:
        end_date = row['Date of Exit'] - timedelta(days=1)

    # Handling tenure in organization
    tenure_in_org = (datetime.now() - row['Date of Joining']).days if not pd.isnull(row['Date of Joining']) else None

    # Handling performance rating
    performance_rating = max(row['Review 1'], row['Review 2']) if not (pd.isnull(row['Review 1']) and pd.isnull(row['Review 2'])) else None

    # Handling engagement score
    engagement_score = max(row['Engagement 1'], row['Engagement 2']) if not (pd.isnull(row['Engagement 1']) and pd.isnull(row['Engagement 2'])) else None

    # Handling last pay raise date
    compensation_dates = [date for date in [row['Compensation 1 date'], row['Compensation 2 date'], row['Date of Joining']] if not pd.isnull(date)]
    last_pay_raise_date = max(compensation_dates).strftime('%Y-%m-%d') if compensation_dates else None

    # Create 15 rows with historical records
    for _ in range(15):
        new_row = {
            'Employee Code': row['Employee Code'],
            'Manager Employee Code': row['Manager Employee Code'],
            'Last Compensation': row['Compensation'],
            'Compensation': row['Compensation'],
            'Last Pay Raise Date': last_pay_raise_date,
            'Variable Pay': 0,
            'Tenure in Org': tenure_in_org,
            'Performance Rating': performance_rating,
            'Engagement Score': engagement_score,
            'Effective Date': row['Date of Joining'].strftime('%Y-%m-%d'),
            'End Date': end_date.strftime('%Y-%m-%d'),
        }
        transformed_data.append(new_row)

# Convert the list of dictionaries to a DataFrame
transformed_df = pd.DataFrame(transformed_data)

# Write the transformed data to a new CSV file
transformed_df.to_csv(output_file, index=False)

print("Transformation complete. Transformed data saved to", output_file)

Transformation complete. Transformed data saved to /content/drive/MyDrive/Colab Notebooks/Historical Data Transformation/output.csv
